In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Change name of dataframe
changes = pd.DataFrame()

In [ ]:
import warnings

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UserWarning)
    changes.plot(figsize=(8, 4))
    plt.ylabel("daily change in returns (%)");

In [ ]:
import arviz as az
import numpy as np
import pymc3 as pm

with pm.Model() as Apple_model:
    change_returns = pm.Data("changes", changes["changes"], dims="Date", export_index_as_coords=True)

    nu = pm.Exponential("nu", 0.1, testval=5.0)
    sigma = pm.Exponential("sigma", 50, testval=0.1)
    s = pm.GaussianRandomWalk("s", sigma=sigma, dims="Date")
    volatility_process = pm.Deterministic("volatility_process", pm.math.exp(-2*s) == 0.5, dims="Date")
    r = pm.StudentT("r", nu=nu, sigma=volatility_process, observed=change_returns, dims="Date")

In [ ]:
with Apple_model:
    trace = pm.sample(2000, init="adapt_diag", return_inferencedata=False)

In [ ]:
with Apple_model:
    az.plot_trace(trace, var_names=["nu", "sigma"]);

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))
changes.plot(ax=ax)
ax.plot(changes.index, 1 / np.exp(trace["s", ::5].T), "C3", alpha=0.03)
ax.set(title="volatility_process", xlabel="time", ylabel="volatility")
ax.legend(["Apple", "stochastic volatility process"], loc="upper right");